A notebook for making a 100-frame movie from 3D dust for Linnea as a test

Two environment variables need to be set in bash: DROPBOX_ROOT, and FFMPEG_ROOT. The DROPBOX_ROOT folder is the path to the top level directory of the dust-holgrams folder shared by Josh. E.g. in bash,

export DROPBOX_ROOT='/Users/catherinezucker/Dropbox/dust-holograms/'

You can also set the path to the FFMPEG executable:

export FFMPEG_PATH='/usr/local/bin'

Ideally, you would set these in your bash_profile profile  (now called zprofile in the latest OS operating systems). Alternatively, if you want to set one of these variables locally in this notebook, you can use:

```
import os
os.environ['DROPBOX_ROOT'] = '/Users/catherinezucker/Dropbox/dust-holograms'
os.environ['FFMPEG_ROOT'] = '/Usr/local/bin'

```

First thing we need to do is figure out how to write some json

In [1]:
import json
import numpy as np
import os
%matplotlib inline
#%matplotlib notebook
from matplotlib import pyplot as plt
from astropy import table

Josh is having some trouble with ```.zprofile``` so:

In [2]:
if (os.environ.keys() != 'DROPBOX_HOME'):
    os.environ['DROPBOX_ROOT'] = '/Users/jegpeek/Dropbox/dust-holograms'
    os.environ['FFMPEG_ROOT'] = '/Users/jegpeek/'

Using a variable called ```run_name``` to record everything we need about the run

In [3]:
def define_paths(run_name):
    if os.path.isdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name)) == False:
        os.mkdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name))
        os.mkdir("{}/{}/frames/".format(os.environ['DROPBOX_ROOT'],run_name))

In [ ]:
arc = False
simpletrack=False

This is the camera properties, which we will fix.

In [4]:

if arc:
    ec = "szyz"
if not arc:
    ec = "rxyz"

t = True
cprops ={
    "projection": "stereographic",
    "step_size": 1,
    "max_dist": 500.0,
    "fov": 180/np.pi,
    "x_pix": 640,
    "y_pix": 480,
    "vmax": "auto",
    "clip_mode": "tanh",
    "fuzzy": t,
    "randomize_angles": t,
    "euler_convention": ec}

In [5]:
print(cprops)

{'projection': 'stereographic', 'step_size': 1, 'max_dist': 500.0, 'fov': 57.29577951308232, 'x_pix': 640, 'y_pix': 480, 'vmax': 'auto', 'clip_mode': 'tanh', 'fuzzy': True, 'randomize_angles': True, 'euler_convention': 'rxyz'}


_In arc mode_ Linnea asked for 100 frames orbiting around a point with equal distance. We'll assume that the middle of these 100 frames is the Sun, and we'll set some sweep out angle in the Galactic Plane.

In [6]:
nframes = 19 # 100 frames
if arc:
    angle_sweep = 180/np.pi # half of this CCW and half CW from the sun

In [7]:
def sweep(xc, yc, angle_sweep, nframes):
    R = np.sqrt(xc**2+ yc**2)
    phi =np.arctan2(yc, xc)*180/np.pi
    dangs = np.linspace(0, angle_sweep, nframes)
    xs = xc - R*np.cos((phi-angle_sweep/2+dangs)*np.pi/180)
    ys = yc - R*np.sin((phi-angle_sweep/2+dangs)*np.pi/180)
    angs = (dangs-angle_sweep/2+phi)*np.pi/180 ## wait are these in RADIANs??
    return xs, ys, angs

In [8]:
def plot_ang(x0, y0, xs, ys, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(x0, y0)
    plt.scatter(0, 0, c='red')
    plt.plot(xs, ys)
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

In [9]:
def plot_track(xts, yts, xc, yc, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(xts, yts)
    plt.scatter(0, 0, c='C1')
    plt.scatter(xc, yc, c='C2')
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

A helper function...

In [10]:
def anglebetween(vec1, vec2):
    return ang

In track mode we'll move from one point to another while pointing at the cloud and keeping to top of the camera perpindicular to the plane that contains the track and cloud

Given the current position of the viewer, the unit direction toward the start of the track, and the cloud position, compute the angles of rotation. The code below is for an arbitary track, and **It does not work yet**.

In [11]:
def determine_angles(v_pos, c_pos, t_hat):
    x_hat = np.array([1, 0, 0])
    y_hat = np.array([0, 1, 0])
    z_hat = np.array([0, 0, 1])
    # the vector to the cloud
    c_vec = c_pos-v_pos
    # the unit vector toward the cloud
    c_hat = c_vec/np.sqrt(np.sum(c_vec**2))
    
    #FIRST
    # the unit vector perp to the plane containing the track and the cloud
    n_OCT = np.cross(c_hat, t_hat) #CHECK ORDER ####FLIPPED
    n_hat_OCT = n_OCT/np.sqrt(np.sum(n_OCT**2))
    #print('n_hat_OCT', n_hat_OCT)
    #print('c_hat', c_hat)
    #print('v_pos', v_pos)
    #print('c_vec', c_vec)
    #print('c_pos', c_pos)
    # find the new orientation of the side of the camera
    v_y_prime = np.cross(n_hat_OCT, x_hat) #CHECK ORDER (correct)
    v_hat_y_prime = v_y_prime/np.sqrt(np.sum(v_y_prime**2))
    # and find the angle to rotate first
    theta = np.arccos(np.dot(v_hat_y_prime,y_hat))
    #print('theta', theta)
    #print('v_hat_y_prime',v_hat_y_prime)
    
    #SECOND
    # the unit vector perp to the plane that contains the rotation 
    #n_phi = np.cross(v_hat_y_prime, x_hat) #CHECK ORDER ###FLIPPED
    #n_hat_phi = n_phi/np.sqrt(np.sum(n_phi**2))
    # the vector being rotated to
    #v_x_prime_prime = np.cross(n_hat_OCT,n_hat_phi) #CHECK ORDER (correct)
    #### OOOPS
    v_x_prime_prime = np.cross(v_hat_y_prime, n_hat_OCT)
    v_hat_x_prime_prime = v_x_prime_prime/np.sqrt(np.sum(v_x_prime_prime**2))
    # and the angle rotated
    phi = np.arccos(np.dot(v_hat_x_prime_prime, x_hat)) ###ORIGINAL
   
    #FINALLY
    #zeta = np.arccos(np.dot(v_hat_x_prime_prime, c_hat)) ####WRONG!!
    
    # This is the fix to deal with the fact that np.arccos(np.dot(x_hat, c_hat)) can never return > 180!! :facepalm:
    # I'd like to make this more generic somehow -- we really want to swap when the direction of "up" 
    ## is opposite the direction of the cross product
    zeta_min = np.arccos(np.dot(v_hat_x_prime_prime, c_hat))
    zeta = zeta_min
    if (c_hat[1] < 0):
        zeta = np.pi*2 - zeta_min
    internal = np.zeros([3, 4])
    internal[:, 0]= c_hat
    internal[:, 1]=n_hat_OCT
    internal[:, 2]=v_hat_y_prime
    internal[:, 3]=v_hat_x_prime_prime
    return theta, phi, zeta, internal
    

In [12]:
def determine_angles_simple(v_pos, c_pos, t_hat):
    x_hat = np.array([1, 0, 0])
    y_hat = np.array([0, 1, 0])
    z_hat = np.array([0, 0, 1])
    # the vector to the cloud
    c_vec = c_pos-v_pos
    # the unit vector toward the cloud
    c_hat = c_vec/np.sqrt(np.sum(c_vec**2))
    #
    zeta_min = np.arccos(np.dot(c_hat, x_hat))
    zeta = zeta_min
    if (c_hat[1] < 0):
        zeta = np.pi*2 - zeta_min
    return zeta

given some inputs build a track

In [13]:
def track(xt1, yt1, zt1, xt2, yt2, zt2, nframes):
    xts = np.linspace(xt1, xt2, nframes)
    yts = np.linspace(yt1, yt2, nframes)
    zts = np.linspace(zt1, zt2, nframes)
    t_vec = np.array([xt2-xt1, yt2-yt1, zt2-zt1])
    t_hat = t_vec/np.sqrt(np.sum(t_vec**2))
    return xts, yts, zts, t_hat

In [14]:
def build_fprops(fprops, cprops, angs, xs, ys, zc):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zc],
          "angles": [angs[i], 0.0, 0.0]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

In [15]:
def build_fprops_track(fprops, cprops, theatas, phis, zetas, xs, ys, zs):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zs[i]],
          "angles": [thetas[i], phis[i], zetas[i]]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

Let's read in a list of molecular clouds and make movies for each one:

In [16]:
clouds = table.Table.read('{}/Holo_Cloud_Targets.csv'.format(os.environ['DROPBOX_ROOT']))
#clouds = table.Table.read('{}/Holo_TEST_Targets.csv'.format(os.environ['DROPBOX_ROOT']))

In [ ]:
makeimages = True
makemovies = True
oldzeta = 0
tl=1
for c in clouds:
    print(c['cloud'])
    run_name = c['cloud'] + 'z=0_Track_640x480'
    define_paths(run_name)
    if arc:
        xs, ys, angs = sweep(c['x'], c['y'], angle_sweep, nframes)
        print(xs, ys, angs)
        plot_ang(c['x'], c['y'], xs, ys, run_name)
        # we make an empty list of frames to which we can append frames
        fprops = []
        allprops = build_fprops(fprops, cprops, angs, xs, ys, np.float(c['z']))
    if (not arc) and (simpletrack):
        xts, yts, zts, t_hat = track(c['y']*tl/2, c['x']*(-1)*tl/2, c['z'], c['y']*(-1)*tl/2, c['x']*(tl/2), c['z'], nframes)
        plot_track(xts, yts, c['x'], c['y'], run_name)
        fprops = []
        thetas = np.zeros(nframes)
        phis = np.zeros(nframes)
        zetas = np.zeros(nframes)
        for j in np.arange(nframes):
            v_pos = np.array([xts[j], yts[j], zts[j]])
            c_pos = np.array([c['x'], c['y'], c['z']])
            zetas[j] = determine_angles_simple(v_pos, c_pos, t_hat)
        allprops = build_fprops_track(fprops, cprops, thetas, phis, zetas, xts, yts, zts)
    if (not arc) and (not simpletrack) :
        #hardcoding a track
        #xts, yts, zts, t_hat = track(c['xt1'], c['yt1'], c['zt1'], c['xt2'], c['yt2'], c['zt2'], nframes)
        xts, yts, zts, t_hat = track(c['y']*tl/2, c['x']*(-1)*tl/2, 0, c['y']*(-1)*tl/2, c['x']*(tl/2), 0, nframes)
        plot_track(xts, yts, c['x'], c['y'], run_name)
        fprops = []
        thetas = np.zeros(nframes)
        phis = np.zeros(nframes)
        zetas = np.zeros(nframes)
        internals = np.zeros([nframes, 3, 4])
        for j in np.arange(nframes):
            v_pos = np.array([xts[j], yts[j], zts[j]])
            c_pos = np.array([c['x'], c['y'], c['z']])
            thetas[j], phis[j], zetas[j], internals[j, :, :] = determine_angles(v_pos, c_pos, t_hat)
            #print(thetas[j]*180/np.pi, phis[j]*180/np.pi, zetas[j]*180/np.pi,(zetas[j]-oldzeta)*180/np.pi )
            oldzeta = zetas[j]
        allprops = build_fprops_track(fprops, cprops, thetas, phis, zetas, xts, yts, zts)

    with open('{}/{}/{}.json'.format(os.environ['DROPBOX_ROOT'],run_name,run_name), 'w') as outfile:
        json.dump(allprops, outfile,indent=2)
    if makeimages:
        os.system("python3 project_frames.py {}/{}/{}.json {}/leike2020_bayestar19_splice.npy {}/{}/frames/{}_{{:05d}}.png"
          .format(os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],os.environ['DROPBOX_ROOT'],run_name, run_name))
    if makemovies:
        os.system("{}/ffmpeg -y -r 30 -start_number 0 -i {}/{}/frames/{}_%05d.png -c:v libx264 -s 600x400 -r 30 -pix_fmt yuv420p {}/{}/{}.mp4"
          .format(os.environ['FFMPEG_ROOT'],os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],run_name, run_name))


Chamaeleon
Loaded specifications for 19 images.


100% (19 of 19) |########################| Elapsed Time: 0:09:34 ETA:  00:00:00

All workers done.
0.5106838424420725
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.007774628117222165
frame 1: vmax = 0.007533695844565955
frame 2: vmax = 0.00825817354517858
frame 3: vmax = 0.007388696284237085
frame 4: vmax = 0.00719197255742256
frame 5: vmax = 0.007780349362605193
frame 6: vmax = 0.008017498441317002
frame 7: vmax = 0.008055286612710915
frame 8: vmax = 0.007619323908413207
frame 9: vmax = 0.007008404297353991
frame 10: vmax = 0.008985140355522163
frame 11: vmax = 0.007833898243698059
frame 12: vmax = 0.008006307721443591
frame 13: vmax = 0.0076154952201177364
frame 14: vmax = 0.007025130389563856
frame 15: vmax = 0.0078811982471816
frame 16: vmax = 0.00702656978018058
frame 17: vmax = 0.007077435990489903
frame 18: vmax = 0.0071199188425380274
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Ophiuchus
Loaded specifications for 19 images.


...

Perseus
Loaded specifications for 19 images.


N/A% (0 of 19) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--Traceback (most recent call last):
  File "project_frames.py", line 270, in <module>
    main()
  File "project_frames.py", line 264, in main
    make_frames_parallel(args.n_processes, args.dustmap, args.output, spec)
  File "project_frames.py", line 136, in make_frames_parallel
    frame_q.join()
  File "/Users/jegpeek/opt/anaconda3/lib/python3.8/multiprocessing/queues.py", line 326, in join
    self._cond.wait()
  File "/Users/jegpeek/opt/anaconda3/lib/python3.8/multiprocessing/synchronize.py", line 261, in wait
    return self._wait_semaphore.acquire(True, timeout)
KeyboardInterrupt
Process Process-1:
Traceback (most recent call last):
  File "/Users/jegpeek/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jegpeek/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Use

0.5106838424420725
Loading map ...
Ray-casting frames ...


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Musca
Loaded specifications for 19 images.
